In [14]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [81]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA]
assets = [:USA_Net, :World_ex_USA]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)



614×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1973-02-28 to 2024-03-31
┌────────────┬────────────┬──────────────┐
│            │ USA_Net    │ World_ex_USA │
├────────────┼────────────┼──────────────┤
│ 1973-02-28 │ -0.0477056 │    0.0975335 │
│ 1973-03-31 │ -0.0116812 │    0.0219335 │
│ 1973-04-30 │ -0.0389935 │   -0.0710148 │
│ 1973-05-31 │ -0.0157639 │     0.017075 │
│ 1973-06-30 │ 0.00151613 │    0.0136391 │
│ 1973-07-31 │    0.03874 │  0.000551845 │
│ 1973-08-31 │ -0.0337141 │   -0.0559634 │
│ 1973-09-30 │  0.0352522 │    0.0182118 │
│     ⋮      │     ⋮      │      ⋮       │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │
│ 2023-10-31 │ -0.0235616 │   -0.0431263 │
│ 2023-11-30 │  0.0895259 │    0.0898198 │
│ 2023-12-31 │  0.0456572 │    0.0532219 │
│ 2024-01-31 │  0.0151977 │   0.00429701 │
│ 2024-02-29 │  0.0518111 │    0.0169284 │
│ 2024-03-31 │  0.0309855 │    0.0331842 │
└────────────┴────────────┴──────────────┘
                          599 rows omitted

In [82]:
returns_summarystats(returns_usd[assets], 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1561,1.95,0.0924,0.4383,-2.8735,-0.6441,-0.209,0.0152,2.2215,0.5922,0.1433
World_ex_USA,0.17,1.8614,0.0767,0.4346,-2.7986,-0.5865,-0.2287,0.0864,1.6006,0.4511,0.1299


In [104]:
hmm = hmm_est(returns_usd[assets], 4)
regime_summary(hmm[1], assets, 12)

Means
Standard deviations
Regimes transition matrix


Regime,USA_Net,World_ex_USA
1,0.225,0.172
2,-0.741,-1.188
3,0.175,0.213
4,-0.552,-0.511


Regime,USA_Net,World_ex_USA
1,0.100,0.117
2,0.328,0.217
3,0.143,0.159
4,0.118,0.149


"Correlations in regime 1"

,USA_Net,World_ex_USA
USA_Net,1.000,0.773
World_ex_USA,0.773,1.000


...

"Correlations in regime 3"

,USA_Net,World_ex_USA
USA_Net,1.000,0.442
World_ex_USA,0.442,1.000


"Correlations in regime 4"

,USA_Net,World_ex_USA
USA_Net,1.000,0.803
World_ex_USA,0.803,1.000


Regime,1,2,3,4
1,0.855,0.000,0.000,0.145
2,0.000,0.249,0.734,0.018
3,0.006,0.000,0.943,0.051
4,0.592,0.231,0.000,0.177


In [105]:
scenarios = simulate_hmm(hmm[1], 2, 180, 100_000)

2×100000×180 Array{Float64, 3}:
[:, :, 1] =
 0.0556121  0.030185  -0.00907385  …   0.0480342  0.00371612  0.0345521
 0.0841319  0.111064  -0.0470702      -0.0049125  0.00669342  0.112564

[:, :, 2] =
 -0.0189689  -0.0202556  -0.0551322  …  -0.0182407    0.00718552  0.066932
 -0.0392926  -0.0513284   0.0503462      0.00854953  -0.013918    0.0372627

[:, :, 3] =
 0.0127827  -0.0010249  -0.00856118  …   0.00479761  -0.0842759  -0.0579932
 0.0150434   0.0290372   0.00926758     -0.0459882   -0.0761101  -0.0148563

;;; … 

[:, :, 178] =
 -0.0196438  -0.0738916  -0.0436303  …  -0.0102106  0.0795719  -0.0538603
 -0.0175583  -0.0815338   0.0256943      0.0291819  0.0599599  -0.0325113

[:, :, 179] =
 0.033648    0.010771   -0.0184118  …  -0.0220462  -0.0395996  -0.0172517
 0.0151542  -0.0227297   0.0107171     -0.0265456  -0.0277839  -0.0518551

[:, :, 180] =
 0.041862   0.0241669   0.0403433  …   0.0108726  0.0554077   0.0374371
 0.0170991  0.00756372  0.0459809     -0.0147422  0.00488408  0

In [106]:
for a in 1:2
    print_percentiles(scenarios[a,:,:], [0.1, 0.3, 0.5, 0.7, 0.9], 12, string(assets))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.0898,-0.1145,-0.1219,-0.1251,-0.1259,-0.1253,-0.1256,-0.1283,-0.1264,-0.1258,-0.125,-0.1266,-0.1246,-0.1246,-0.1258
0.3,0.0421,0.0246,0.02,0.0184,0.0162,0.0169,0.0167,0.0175,0.0161,0.0175,0.0163,0.0179,0.0166,0.0163,0.0164
0.5,0.1261,0.1099,0.1059,0.1038,0.1036,0.1024,0.103,0.1036,0.1032,0.1039,0.1032,0.1045,0.1039,0.1023,0.1038
0.7,0.2073,0.1901,0.1861,0.1845,0.1834,0.1826,0.1837,0.1834,0.1825,0.1843,0.183,0.1842,0.1833,0.1828,0.1835
0.9,0.3223,0.3033,0.2964,0.2945,0.2929,0.2912,0.2907,0.2926,0.2924,0.2949,0.2917,0.2926,0.2927,0.2915,0.2925


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.1029,-0.1503,-0.1614,-0.167,-0.1668,-0.1672,-0.1683,-0.1681,-0.1669,-0.1688,-0.1684,-0.1687,-0.1673,-0.1667,-0.1673
0.3,0.0464,0.0046,-0.0069,-0.0137,-0.0134,-0.0147,-0.0133,-0.0141,-0.0142,-0.0141,-0.015,-0.0129,-0.0138,-0.0138,-0.0139
0.5,0.1434,0.1021,0.0889,0.0836,0.0829,0.0816,0.0834,0.0818,0.0814,0.0821,0.0799,0.0833,0.0818,0.0815,0.0824
0.7,0.2375,0.1947,0.1817,0.1759,0.1743,0.1725,0.1732,0.1727,0.1737,0.1748,0.1726,0.1741,0.1733,0.173,0.1736
0.9,0.3701,0.3299,0.3152,0.3087,0.3058,0.3039,0.305,0.3061,0.3065,0.3078,0.3052,0.304,0.3049,0.305,0.3057


In [108]:
#weights = [0.62, 0.26, 0.12]
weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(expm1.(acwi),[0.02, 0.1, 0.3, 0.5, 0.7, 0.9, 0.98], 12)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0.02,-0.231,-0.2604,-0.271,-0.2704,-0.273,-0.2753,-0.2727,-0.2728,-0.2732,-0.2763,-0.2732,-0.2725,-0.2723,-0.2669
0.1,-0.0739,-0.0989,-0.1062,-0.1112,-0.1122,-0.1111,-0.1121,-0.1123,-0.112,-0.1123,-0.112,-0.1123,-0.1117,-0.1114
0.3,0.0545,0.0344,0.0294,0.0248,0.0242,0.0242,0.0239,0.025,0.0236,0.0244,0.0244,0.0251,0.0245,0.0236
0.5,0.1362,0.1179,0.1119,0.1092,0.1082,0.1073,0.1078,0.1083,0.108,0.1089,0.1084,0.1091,0.1088,0.1079
0.7,0.2155,0.1963,0.1899,0.1875,0.1863,0.1856,0.1862,0.1872,0.1855,0.1875,0.1866,0.1872,0.1868,0.1864
0.9,0.3249,0.3052,0.2987,0.2949,0.2935,0.293,0.2929,0.2948,0.2937,0.2961,0.294,0.2939,0.2949,0.2941
0.98,0.4353,0.4138,0.4047,0.4028,0.401,0.4019,0.4015,0.4008,0.4029,0.4046,0.4022,0.4023,0.4045,0.4017
